In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [5]:
df = pd.read_csv("data.csv")

In [19]:
df['GENDER'] = df['GENDER'].map({'M': 1, 'F': 0})
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})

In [20]:
X = df.drop("LUNG_CANCER", axis=1)
y = df["LUNG_CANCER"]


In [21]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
u = pd.DataFrame(X_scaled)
u.head()
print(u.iloc[2,:])

0    -1.049781
1    -0.448107
2    -1.135292
3    -1.150351
4    -0.996769
5     0.996769
6    -1.009756
7     0.696833
8    -1.120480
9     0.892475
10   -1.120480
11    0.852207
12    0.748736
13   -0.940291
14    0.892475
Name: 2, dtype: float64


In [23]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


In [25]:
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [26]:
model = DeepNN(X_tensor.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)  # L2 regularization
print(X_tensor.shape[1])

15


In [27]:
# Train the model
epochs = 50
for epoch in range(epochs):
    model.train()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 10/50, Loss: 0.6476
Epoch 20/50, Loss: 0.6323
Epoch 30/50, Loss: 0.6178
Epoch 40/50, Loss: 0.6040
Epoch 50/50, Loss: 0.5900


In [28]:
print(X_test[2])

tensor([ 0.9526,  1.2598, -1.1353, -1.1504, -0.9968, -1.0032,  0.9903, -1.4351,
         0.8925, -1.1205,  0.8925,  0.8522,  0.7487,  1.0635,  0.8925])


In [2]:
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
def load_model(input_dim):
    
    saved_model = DeepNN(input_dim)
    saved_model.load_state_dict(torch.load("lung_cancer_model.pth"))
    saved_model.eval()


    return saved_model

model = load_model(15)

In [6]:
X_t = pd.read_csv("test.csv")
y_t = pd.read_csv("y_test.csv")

In [8]:
X_test = torch.tensor(X_t.values, dtype=torch.float32)
y_test = torch.tensor(y_t.values, dtype=torch.float32).view(-1, 1)

In [ ]:
predicted_classes=[]

In [ ]:
%%time
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test)
    global predicted_classes
    predicted_classes = (y_test_pred > 0.5).float()
    



CPU times: total: 0 ns
Wall time: 1.94 ms


In [12]:
acc = accuracy_score(y_test, predicted_classes)
print("\n✅ Test Accuracy:", round(acc * 100, 2), "%")
print("\n📊 Classification Report:\n", classification_report(y_test, predicted_classes))


✅ Test Accuracy: 96.77 %

📊 Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       0.97      1.00      0.98        60

    accuracy                           0.97        62
   macro avg       0.48      0.50      0.49        62
weighted avg       0.94      0.97      0.95        62



c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [30]:
test_df = pd.DataFrame(X_test.numpy())
test_df.to_csv("test.csv",index=False)

In [33]:
y_test_df = pd.DataFrame(y_test.numpy())
y_test_df.to_csv("y_test.csv",index=False)

In [31]:
torch.save(model.state_dict(), "lung_cancer_model.pth")

In [32]:
saved_model = DeepNN(X_tensor.shape[1])
saved_model.load_state_dict(torch.load("lung_cancer_model.pth"))
saved_model.eval()
for idx, layer in enumerate(saved_model.model):
    if isinstance(layer, nn.Linear):
        print(f"Layer {idx} - Linear")
        print(f"  Weights shape: {layer.weight.shape}")
        print(f"  Weights:\n{layer.weight.data}\n")
        print(f"  Biases shape: {layer.bias.shape}")
        print(f"  Biases:\n{layer.bias.data}\n")

Layer 0 - Linear
  Weights shape: torch.Size([64, 15])
  Weights:
tensor([[-4.1219e-03, -4.0772e-03,  5.2254e-02,  3.1771e-03, -2.2988e-03,
         -3.9628e-02,  4.0029e-02, -4.4856e-03,  8.2830e-03,  1.3265e-02,
          4.5049e-03,  5.8502e-03,  1.3710e-03, -1.2033e-02,  9.2761e-03],
        [-2.4670e-03,  1.0034e-03, -8.1586e-02,  4.9639e-03, -2.8503e-03,
          4.6915e-03, -3.5537e-04, -5.8303e-03,  1.3055e-02, -1.6729e-04,
          5.5537e-02, -1.8748e-02,  2.9626e-03, -2.3276e-03, -5.5283e-02],
        [ 1.1250e-03, -8.1622e-04,  7.6855e-03,  9.6560e-04,  9.1170e-03,
         -6.6143e-03,  6.7219e-03,  3.4067e-03, -6.1357e-03,  8.7705e-03,
         -3.4825e-03, -1.9635e-02, -1.8824e-03,  2.5263e-02,  5.8122e-03],
        [-2.9898e-02,  1.8160e-03, -5.9474e-03,  1.5897e-03,  2.4387e-02,
         -1.1279e-03, -3.4875e-03,  2.1391e-03, -2.1904e-03, -1.2336e-04,
          9.1930e-03,  7.0999e-03, -1.2331e-03,  4.5436e-03,  5.8093e-03],
        [-7.5994e-05,  9.1475e-03, -8.2576